

<img align="left" width="30%" style="padding-right:10px;" src="../Images/Ccom.png">

___
# Computational Problem Set - Being a VGNSS Receiver

## Step 3 Computing Satellite Vehicle Ephemeris

## Semme J. Dijkstra

<div class="footer">
    <br><p>VGNSS Step 3: Semme J. Dijkstra February 5 20, 2020
    </p>
</div> 


In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np

from numpy import cos,pi,sin,pi,arccos, tan, arctan
from datetime import datetime, timedelta
from pathlib import Path

vgnss_path=Path('../') # Get the path to the folder containing the mycode folder

print(vgnss_path.resolve())
sys.path.append(str(vgnss_path.resolve())) # add the folder to the list of paths 

from mycode.gnss import GNSS
from mycode.sp3 import SP3
# from mycode.lsq import LSQ
# from mycode.ephemeris import Ephemeris

Convert code to .py file

___
## 3 Computing Satellite Vehicle Ephemeris

### Objective:

Get familiar with ephemeris calculations, both using broadcast ephemeris and sp3 files

### Assignment:

Get the broadcast data from the NASA website and calculate the ephemeris for the epoch defined by you in step 1

### Deliverable:

Completed steps of this assignment


___
# 3.0 Get Ephemeris Data

### Objective:

Get an understanding that there are multiple methods of retrieving ephemeris data for a given epoch 

### Assignment:

Describe the difference between methods of ephemeris data through answering a series of questions

### Deliverables:

- Answers to questions asked 

___

There are three methods to calculate ephemeris for a given epoch. 
    
    1 Calculate satellite coordinates from Almanac data
    2 Calculate satellite coordinates from broadcast data
    3 Calculate satellite coordinates from post processed ephemeris data

For GPS the first 2 methods rely on Keplerian analysis of the satellite orbits. That is, using a set of parameters given in either an almanac or the GPS broadcast messages use Kepler's law of planetary motion to calculate the ephemeris of the satellite vehicles at the time of interest. For the third method we simply interpolate the satellite vehicle positions between epochs for which the ephemeris are known in ECEF cooordinates. For this we will us Lagrange polynomials. For GLONASS the same techniques are used, with the difference that the broadcast ephemeris are transmitted as ECEF coordinates rather than orbital parameters.

___
### 3.0.0 

In your own words describe the difference between almanac data and broadcast ephemeris data - hint search for 'What is the difference  between almanac and broadcast data for GPS'

Fill in the answer below as follows:

    q_3_0_0 = '...'
    
Make sure that the answer is *not* commented out

In [ ]:
q_3_0_0 = 'The difference between almanac and broadcast data is ...'

print( q_3_0_0)

___
### 3.0.1

Given your answer above explain why it is beneficial to collect all broadcast data and make them available for download in a single file for a given day
Fill in the answer below as follows:

    q_3_0_0 = '...'
    
Make sure that the answer is *not* commented out

In [ ]:
q_3_0_1 = 'Providing brdc files is beneficial because...'

print( q_3_0_1)

___
### 3.0.2

Given the nature of brdc files, are the ephemeris that you calculate from the parameters contained within considered 'ultra-rapid', 'rapid', or 'precise'

    q_3_0_2 = '...'
    
Make sure that the answer is *not* commented out

In [ ]:
q_3_0_2 = 'Ephemeris calculated using brdc files are considered to be ... because ...'

print( q_3_0_2)

___
### 3.0.3

For the third method of ephemeris calculation we use SP3 files. These SP3 files are fundamentally different than the almanac files that contain Keplerian parameters. As a result we use interpolation rather than Orbital modeling for the determination of the satellite ephemeris at the time of interest.

Can you think of a reason that SP3 files use ECEF coordinates, whereas almanac files use Keplerian elements? 

    q_3_0_3 = '...'
    
Make sure that the answer is *not* commented out

In [ ]:
q_3_0_3 = 'SP3 files use ECEF coordinates as opposed to orbital parameters because ...'

print( q_3_0_3)


The code cell below retrieves both the broadcast and SP3 ephemeris files available for a given epoch i.e., it does the steps that you did manually in step 2 for you. 

In [ ]:
# Define the path where to find and place data
data_path = os.path.join(vgnss_path, "mydata")
data_path = os.path.abspath(data_path)

# Create a GNSS object
gnss = GNSS( data_path)

# Run VGNSS_1 to get the right epoch data
# %run VGNSS_1.ipynb

gnss.add_next_epoch_ephemeris(2020, 3, 23, 15, 0, 0, 0)

___
### 3.0.4

Rerun the code cell above using an epoch differing by at least a day from any epoch you used before (and different from your step 2) and capture the output text

In [ ]:
q_3_0_4 = ''

print( q_3_0_4)

___
### 3.0.5

Rerun the code cell again with the same epoch as for 3.0.4 and capture the output text again

You will notice that there is less text output for the 2nd attempt - do you understand why? (if not, look at the `GNSS.add_next_epoch_ephemeris()` and the contents of your `mydata` folder).

In [ ]:
q_3_0_5 = 'The method GNSS.add_next_epoch_ephemeris() provides less output because ...'

print( q_3_0_5)

___
### 3.0.6

If you pick a different epoch on the same date should additional files be downloaded?

In [ ]:
q_3_0_6 = 'The method GNSS.add_next_epoch_ephemeris() (does/does not) download extra files because ...'

print( q_3_0_6)

___
## 3.1  Keplerian Motion


### Objective:

Get an understanding that there are multiple methods of computing ephemeris data for a given epoch 

### Assignment:

Describe the difference between methods of ephemeris determination for a given epoch

### Deliverable:

Provide the answers requested below.


Keplerian motion is the motion of **satellites** in their orbits. Satellites can be artificial and placed in an orbit or can be celestial. In both cases satellite motion is the motion of one object around another and is strongly dependent on the gravitational attraction between them. Johannes Kepler developed three laws of planetary motion around the sun in the early 17th century. At the end of the 17th century Newton showed that Kepler's laws are consistent with the law of universal gravitation and laws of motion. This makes that the Kepler's laws may be used to model satellite orbital motion with sufficient accuracy to determine satellite positions. Note that for timing relativistic effects come into play, which is a subject outside the scope of this course.

<img align="left" width="100%" style="padding-right:10px;" src="../Images/classical-orbital-elements-n.jpg">
image source : https://www.slideserve.com/cyma/topic-5-intro-to-space-orbits-enabling-objectives-5-1-describe-space-operations-powerpoint-ppt-presentation


A generalized version of Kepler's laws states that:

    1 The orbit of a satellite around a body is an ellipse with the body at
      one of the two foci.
    2 A line segment joining the body and the satellite sweeps out 
      equal areas during equal intervals of time.
    3 The square of the orbital period of a satellite is directly proportional 
      to the cube of the semi-major axis of its orbit.
      

      
The first two of Kepler's laws may be combined to calculate the position of satellites for a given epoch. The combination of the first two laws creates Kepler's equation that allows the position determination of the satellite in polar coordinates with their origin at the center of the body that they orbit. In the case of **GPS** that is the center of the reference frame defined by **WGS84**. The position of the satellite is then a function of time since being in a given location of the orbit. The reference used by Kepler's laws is the **perigee**, that is the location at which the satellite is closest to earth. The angle $\nu$ describing the position is known as the **true anomaly** $\nu$ and is the angle from the perigee increasing in the direction of the satellite motion.

Using this we can calculate the position of the satellite vehicle in polar coordinates by:

    1 Computing the true anomaly 𝜈 describing the location of the satellite relative to earth.
    2 Computing the orbit radius for this location

We can then find the `ECEF` coordinates by transforming the polar coordinates to Cartesian coordinates by relating the orbit parameters to the WGS84 system. These steps are divided in a number of smaller steps. All of these steps are implemented in the `gnss.get_single_epoch_ephemeris_from_gps_nav()` method. Note that the implementation of the method is based on the paper [Computing satellite velocity using the broadcast ephemeris](../Documentation/Remondi2004_Article_ComputingSatelliteVelocityUsin.pdf) by Ben Remondi(2004). In the mean time an improved modeling method has been published which is finding general acceptance; This method is found in the paper [Computing GPS Satellite Velocity and Acceleration from the Broadcast Navigation Message](../Documentation/GPS-SV-velocity-and-acceleration.pdf) by Thompson, et al. (2019) - if you feel up to it you may implement this version instead. 

___
### 3.1.0 Establishment of the WGS84 System Parameters

The first step is to set the parameters that implement the definition of the coordinate system. `WGS84` is the system used for GPS and the parameters of relevance to positioning satellite vehicles (**sv**s) are the `GM` for the gravitational constant for the Earth (in literature indicated as $\nu$ or GM) and `w_e` as the rotation rate $\omega$.

$$ GM = \nu = 3.986005\cdot10^{14} $$
$$ \omega = 7.2921151467\cdot10^{-5} $$



### 3.1.1 Allocation of Memory for Ephemeris Data

The `get_single_epoch_ephemeris_from_gps_nav()` method will return ephemeris data. In this step we will be using this method. The return data `eph` consist of matrix with the number of rows matching the number of satellites for which data are available. The data used in this step are from the GPS broadcast ephemeris, that is the data that the GPS svs transmit while in orbit. Given that we are dealing with a virtual receiver we do not have the capability to actually receive these data. Fortunately these data are however stored at the various gnss data centers around the world, including the NASA repository used for this assignment. The `GNSS.add_next_epoch_ephemeris()` automatically retrieves these data for GPS and stores them in a `RINEX_nav` object. The `RINEX_nav` class was specifically created to be able to read and hold the broadcast ephemeris data that is disseminated in the *Receiver Independent Exchange Format* (**RINEX**) navigation files. In case of GPS broadcast data these are the Keplerian elements needed to calculate the position of the svs. Much like the precise ephemeris files you analyzed in step 2 the data is organized in epochs and given for each PRN (You will find a RINEX format description in the documentation). You can determine the number of satellites by counting the PRN identifiers for the `RINEX_nav` object `gps_nav` holding the data.

Note: Please explore the documentation folder for the VGNSS assignment -- you will find many useful files there


<div class="footer">
    <br><p>VGNSS Step 0: Semme J. Dijkstra December 20, 2019</p>
</div> 

In [ ]:
# Print the number of satellites contained in the broadcast ephemeris
print('Number of satellites for which broadcast ephemeris are available: '+str(len(gnss.eph_gps_nav[-1])))

___
### 3.1.1.0

Is the number of satellites listed above all the GNSS satellites for which broadcast ephemeris are available? Can you suggest a clearer output message?

In [ ]:
q_3_1_1_0 = 'I think the output message should be ... because ...'

print( q_3_1_1_0)


___
### 3.1.2 Get the Ephemeris Data for Satellites

For the day of our epoch we will cycle through all of the satellites whose data are contained in the broadcast ephemeris file. The `RINEX_nav` object `gps_nav` holds a list of records containing the orbit parameters for each satellite at different epochs, these records are contained in the list `gps_nav.records`. These records are `structured arrays` which are data records in which the fields may be addressed by name. You will notice that in the file `rinex_nav.py` there is the definition of the `gps_d_type`. This `gps_d_type` is then the structure that can hold the data associated to the broadcast ephemeris. If you look at the definition of the RINEX navigation data structure (included in the documents section) you will see that the parameters match the contents of a GPS navigation message.

If for example you want to see what the PRN is for the last record contained in `gps_nav` you would use:

    gps_nav.records[-1]['prn']

In the code cell below you will retrieve the last `GNSS.gps_nav` object that the `gnss` object holds.

In [ ]:
# Get the RINEX_nav object holding the broadcast ephemeris for the current epoch
gps_nav = gnss.gps_nav[-1]

# Get the PRN of the last record in the broadcast ephemeris for the current epoch
print( 'The last record in the broadcast ephemeris for the current epoch are for PRN: '+ \
      str(gps_nav.records[-1]['prn']))

___
### 3.1.1.1

The code cell below shows the epoch time for each `gps_nav` record - update it so that it also shows the PRNs ephemeris data available at that epoch

In [ ]:
for rec in gps_nav.records:
    print(str(rec['t_oe'])...)
    

    Sample output: 
        518400.0    PRN: 1
        518400.0    PRN: 2
        518400.0    PRN: 3
        518400.0    PRN: 4
        518400.0    PRN: 5
        ...

___
### 3.1.1.2

Looking at the above: are there broadcast data available for each epoch for each satellite? Do you think orbit parameters are stable enough to allow interpolation over larger periods - why?

In [ ]:
q_3_1_1_2 = '... I think orbit parameters are ... because ...'

print( q_3_1_1_2)

You can also subset the records by the contents of the records. For example you can find all the records with PRN 25 by using 

    gps_nav.records[ gps_nav.records['prn'] == 25]

the `RINEX_nav` keeps track of all the **PRN**s for which data are available in the broadcast ephemeris (peruse the code if you want to understand how this is done). You can determine the number of satellites by determining the length of the list of svs for which ephemeris are available (see code cell below)

In [ ]:
# Get a subset of records for sv with PRN 25
prn_25_recs = gps_nav.records[ gps_nav.records[...] == ...]
# To demonstrate that this worked show the prn code for the records just found

print( prn_25_recs['prn'])
# Also show the time of ephemeris for these recods (seconds since )

print("t_oe: "+str(prn_25_recs[...]))
# Print a list of all the PRNs for which PRN are available

print('File '+gnss.eph_gps_nav_filenames[-1]+' contains records for PRNs:'+str(gps_nav....))

     Sample output:
        [25 25 25 25 25 25 25 25 25 25 25 25 25]
        t_oe: [518400. 525584. 532800. 540000. 547200. 554400. 561600. 568800. 576000.
         583200. 590400. 597600. 604768.]
         File brdc0810.20n contains records for PRNs:[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 
         12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 14]


___
### 3.1.1.3

Update the code cell below to determine the number of satellites for which your broadcast file holds records and assign it to n_PRNS - you may look at the last line 

In [ ]:
# 3.1.1.3

n_PRNS = len(gps_nav.sat_prns)

print( 'The value of n_PRNS is: ' ...)

    Sample Output:
        The value of n_PRNS is: (Make sure that the right number is displayed e.g. 32)

The code cell below uses both the broadcast orbit data as well as the ephemeris data contained in SP3 files for the same epoch. You would expect the data to be very similar, but slightly different.

<div class="footer">
    <br><p>VGNSS Step 0: Semme J. Dijkstra December 20, 2019</p>
</div> 

In [ ]:
data_path = os.path.join(vgnss_path, "mydata")
data_path = os.path.abspath(data_path)
gnss = GNSS( data_path)

gnss.add_next_epoch_ephemeris(2008,7,23,2,1,45,0)

ind = gnss.eph_gps_nav[-1][:,0] == 20 
print(gnss.eph_gps_nav[-1][ind,0])
print(gnss.eph_gps_nav[-1][ind,1])
print(gnss.eph_gps_nav[-1][ind,2])
print(gnss.eph_gps_nav[-1][ind,3])

ind = gnss.eph_gps_sp3[-1][:,0] == 20 
print(gnss.eph_gps_sp3[-1][ind,0])
print(gnss.eph_gps_sp3[-1][ind,1])
print(gnss.eph_gps_sp3[-1][ind,2])
print(gnss.eph_gps_sp3[-1][ind,3])

if gnss.gnss_weeks[-1] < 1300:
    raise RuntimeError( 'epoch predates GLONASS, please pick a later date')
if not gnss.eph_gps_sp3_filenames[-1][0:3] == 'igs':
    raise RuntimeError( gnss.eph_gps_sp3_filenames[-1][0:3])



___
### 3.1.1.4

Given the nature of the broadcast files and the SP3 files (how are they created?) which estimate of the ephemeris for the satellite `prn` at the epoch is more accurate i.e., which is preferred for usage?

In [ ]:
q_3_1_1_4 = 'The (brdc/SP3) ephemeris results are preferred because ...'

print( q_3_1_1_4)

    Sample output:
        The (brdc/SP3) ephemeris results are preferred because ...

## 3.2 Ephemeris from SP3 Files Using Lagrange Polynomials

The following is based on Wikipedia on February 19, 2020
___

Given a set of $k + 1$ data points

$$(x_0,y_0), … ,(x_j,y_j), … ,(x_k,y_k)$$ 

where no two $x_j$ are the same, the **interpolation polynomial in the Lagrange form** is a linear combination

$$L(x):=\sum\limits_{j=0}^k y_j\cdot ℓ_j ( x )$$

of **Lagrange basis polynomials**

$$ℓ_j(x) :=\prod_{\begin{smallmatrix}0\leq m\leq k\\m\neq j\end{smallmatrix}}\frac{x−x_m}{x_j−x_m} = \frac{(x−x_0)}{(x_j−x_0)} ⋯ \frac{(x−x_{j−1})}{(x_j−x_{j−1})}\cdot \frac{(x−x_{j+1})}{(x_j−x_{j+1})} ⋯ \frac{(x−x_{k})}{(x_j−x_{k})}$$ 

where $0 ≤ j ≤ k$. Note how, given the initial assumption that no two $x_j$ are the same, then (when $m ≠ j$)$x_{j}-x_{m}\neq 0$, so this expression is always well-defined. This means that pairs $x_{i}=x_{j}$ with $y_{i}\neq y_{j}$ are not allowed, which is desirable as no interpolation function $L$ such that $y_i = L(x_i)$ would exist; a function can only get one value for each argument $x_i$. On the other hand, if also $y_i = y_j$, then those two points would actually be one single point, but still lead to failure in the interpolation method due to the division by zero.

For all $i ≠ j,\ell_{j}(x)$ includes the term $(x−x_ i)$ in the numerator, so the whole product will be zero at $x=x_i$:

$$∀(j≠i):ℓ_j(x_i)=\prod_{\begin{smallmatrix}0\leq m\leq k\\m\neq j\end{smallmatrix}}\frac{x_i−x_m}{x_j−x_m} = \frac{(x_i−x_0)}{(x_j−x_0)} ⋯ \frac{(x_i−x_{i})}{(x_j−x_{i})} ⋯ \frac{(x_i−x_{k})}{(x_j−x_{k})} = 0$$ 

On the other hand:

$$ℓ_j(x_j) :=\prod_{\begin{smallmatrix}0\leq m\leq k\\m\neq j\end{smallmatrix}}\frac{x_i−x_m}{x_j−x_m} = 1$$

In other words, all basis polynomials are zero at $x = x_j$, except $ℓ_j\cdot (x)$, for which it holds that $ℓ_j(x_j) = 1$, because it lacks the $(x−x_j)$ term.

It follows that $y_j\cdot ℓ_j\cdot(x_j)=y_j$, so at each point $x_j,L(x_j)=y_j+0+0+⋯+0=y_j$, showing that $L$ interpolates the function exactly.
___

Note that the formulation of the **Lagrange basis polynomials** implies that to be able to fit functions of order $n$ we need $n+1$ samples i.e., for a linear function we require 2 samples, a quadratic function 3 samples, etc.

Also note that as more samples are used the higher the order of the **Lagrange basis polynomials** is. This may lead to numerical issues for large sampling sets rendering the results to be unstable. For larger data sets it is therefore often better to creating a moving window for the interpolation. The size of the window can be bigger if the data is smoother. For GNNS ephemeris data we often use a window of 6 samples, that is we use a polynomial fit order of $6-1=5$
___

### Example

Interpolate $f(x) = x^2$ over the range $\{x\in\Re|0 \leq x \leq 3\}$ given the three points:

$\vec x = \begin{bmatrix}1\\2\\3\end{bmatrix} \quad f(\vec x) = \begin{bmatrix}1\\4\\9\end{bmatrix}$

This leads to the Lagrange basis polynomials

$l_0(x) = \dfrac{x-x_1}{x_0 - x_1}\cdot\dfrac{x-x_2}{x_0-x_2} =  \dfrac{x-2}{1 - 2}\cdot\dfrac{x-3}{1-3} $

$l_1(x) = \dfrac{x-x_0}{x_1 - x_0}\cdot\dfrac{x-x_2}{x_1-x_2} =  \dfrac{x-1}{2 - 1}\cdot\dfrac{x-3}{2-3} $

$l_2(x) = \dfrac{x-x_0}{x_2 - x_0}\cdot\dfrac{x-x_1}{x_2-x_1} =  \dfrac{x-1}{3 - 1}\cdot\dfrac{x-2}{3-2} $

Thus the interpolating polynomial:

$L(x):=\sum\limits_{j=0}^k y_j\cdot ℓ_j(x) = f(x_0)\cdot l_0 + f(x_1)\cdot l_1 + f(x_2) \cdot l_2 \Rightarrow$

$L(x) = 1\cdot\dfrac{x-2}{1 - 2}\cdot\dfrac{x-3}{1-3}+4\cdot \dfrac{x-1}{2 - 1}\cdot\dfrac{x-3}{2-3}+9\cdot\dfrac{x-1}{2 - 1}\cdot\dfrac{x-2}{3-2} $

Although it would be relatively easy to implement Lagrange interpolation ourselves we will make use of the SciPy implementation of Lagrange Polynomials. In the code cell below you will see the example above and an additional example for an order 3 function


In [ ]:
from scipy.interpolate import lagrange
from numpy.polynomial.polynomial import Polynomial

print("Example 1) f(x) = x**2")
x = np.array([1, 2, 3])
f_x = x**2
print("x    = "+str(x))
print("f(x) = "+str(f_x))
# get the tensor of polynomials
poly = lagrange(x, f_x)
t = Polynomial(poly).coef
# print('The polynomial tensor: '+str(t))
print('The polynomial value at x = 2: '+str(np.polyval(t,2)))
print('The polynomial value at x = 3: '+str(np.polyval(t,3)))
print('The polynomial value at x = 2.4: '+str(np.polyval(t,2.4)))
print('The actual value at x = 2.4: '+str(2.4**2))

print(" ")
print("Example 2) f(x) = x**3")
x = np.array([0, 1, 2, 3])
f_x = x**3
print("x    = "+str(x))
print("f(x) = "+str(f_x))
# get the tensor of polynomials
poly = lagrange(x, f_x)
t = Polynomial(poly).coef
# print('The polynomial tensor: '+str(t))
print('The polynomial value at x = 2: '+str(np.polyval(t,2)))
print('The polynomial value at x = 3: '+str(np.polyval(t,3)))
print('The polynomial value at x = 2.4: '+str(np.polyval(t,2.4)))
print('The actual value at x = 2.4: '+str(2.4**3))

print(" ")
print("Example 3) f(x) = x**3 + 2*x + 3")
x = np.array([0, 1, 2, 3])
f_x = x**3+2*x+3
print("x    = "+str(x))
print("f(x) = "+str(f_x))
# get the tensor of polynomials
poly = lagrange(x, f_x)
t = Polynomial(poly).coef
# print('The polynomial tensor: '+str(t))
print('The polynomial value at x = 2: '+str(np.polyval(t,2)))
print('The polynomial value at x = 3: '+str(np.polyval(t,3)))
print('The polynomial value at x = 2.4: '+str(np.polyval(t,2.4)))
print('The actual value at x = 2.4: '+str(2.4**3+2*2.4+3))



___
#### 3.2.0 

The code cell below prints the ephemeris data for GPS satellite PRN 20 at the epoch defined in the call to the method `GNSS.add_next_epoch_ephemeris()`. Can you think of an approach to verify that the results are correct?

In [ ]:
gnss.add_next_epoch_ephemeris(2008,7,23,2,1,51,17)

ind = gnss.eph_gps_sp3[-1][:,0] == 20 
print(gnss.eph_gps_sp3[-1][ind,0])
print(gnss.eph_gps_sp3[-1][ind,1])
print(gnss.eph_gps_sp3[-1][ind,2])
print(gnss.eph_gps_sp3[-1][ind,3])

In [ ]:
q_3_2_0_0 = 'I would verify the correctness of the results by ...'

print( q_3_2_0_0)

    Sample output:
        I would verify the correctness of the results by ...